# 作业三（列生成算法)
# author:陈伟钊

# 钢管数量最小

In [3]:
from gurobipy import *
import numpy as np

typesDemand = [2.9,2.1,1.5]           # 需求长度
quantityDemand = [100,100,100]        # 需求数量
length = 7.4                          # 钢管长度

# 模型
mainModel = Model('Main Model')
subModel = Model('SubModel')

# 变量
x = mainModel.addVars(len(typesDemand),obj=1,vtype='c',name='x')

fangan = [
            [2,0,1],
            [0,3,0],
            [0,0,4]
        ]

# 约束
mainModel.addConstrs((x[0]*fangan[0][j] + x[1]*fangan[1][j] + x[2]*fangan[2][j]>= quantityDemand[j]\
                      for j in range(len(typesDemand))),name='mainCon')

# 求解主问题
subModel.Params.OutputFlag = 0
mainModel.optimize()

# 获取对偶值
Dual = mainModel.getAttr(GRB.Attr.Pi,mainModel.getConstrs())

# 添加子问题变量
X = subModel.addVars(len(typesDemand),obj=Dual,vtype='I',name='X')

# 添加子问题约束
subModel.addConstr(X.prod(typesDemand)<=length,name='subCon')

# 求解子问题
subModel.setAttr(GRB.Attr.ModelSense,-1) # 最大化
subModel.Params.OutputFlag=0
subModel.optimize()

# 根据 reduced cost 添加进基列
while subModel.objVal>1:

    # 获取切割方案
    columnCoeff=subModel.getAttr('X',subModel.getVars())
    
    # 保存切割方案
    tmp_list_save = []
    for i in range(len(typesDemand)):  
        tmp_list_save.append(int(subModel.getVars()[i].x))
    fangan.append(tmp_list_save)
   
    # 定义列
    column=Column(columnCoeff,mainModel.getConstrs())
    
    # 添加列
    mainModel.addVar(obj=1.0,column=column)
    
    # 求解更新后的主问题
    mainModel.Params.OutputFlag=0
    mainModel.optimize()
    
    # 更新对偶值
    Dual=mainModel.getAttr(GRB.Attr.Pi,mainModel.getConstrs())
    
    # 重新求解子问题(修改目标函数系数)
    for i in range(len(typesDemand)):
        X[i].obj = Dual[i]
    subModel.Params.OutputFlag=0
    subModel.optimize()
    
# 将主模型变量类型改成整型
for v in mainModel.getVars():
    v.setAttr('vtype',GRB.INTEGER)
    
# 求解主模型
mainModel.Params.OutputFlag=0
mainModel.optimize()

#输出
print('最少使用钢材数为：',mainModel.objVal)
for i in range(len(mainModel.getVars())):
    v=mainModel.getVars()[i]
    if v.x>0:
        print('方案{}切割方案为:{}，切割次数为{}'.format(i+1,fangan[i],v.x))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 4 nonzeros
Model fingerprint: 0xa22bc830
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.5833333e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.583333333e+01
最少使用钢材数为： 90.0
方案1切割方案为:[2, 0, 1]，切割次数为40.0
方案4切割方案为:[0, 2, 2]，切割次数为30.0
方案5切割方案为:[1, 2, 0]，切割次数为20.0
